In [1]:
! pip install trl
! pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 376.2/376.2 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.8/494.8 kB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 80.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 95.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from huggingface_hub import login
import os
import pandas as pd
import torch
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
    LlamaForCausalLM
)

import os
from trl import SFTTrainer, SFTConfig
import json
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset, Dataset
import multiprocessing
from peft import get_peft_model

In [ ]:
hf_token = "<Your token here>"

if hf_token is None:
    raise EnvironmentError("HUGGINGFACE_TOKEN is not set in the environment variables.")
login(hf_token)

In [4]:
checkpoint = "HuggingFaceTB/SmolLM-360M"
tokenizer = AutoTokenizer.from_pretrained(checkpoint, padding=True, truncation=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/831 [00:00<?, ?B/s]

In [5]:
print(tokenizer.pad_token, tokenizer.eos_token, tokenizer.unk_token)
print(tokenizer.pad_token_id, tokenizer.eos_token_id, tokenizer.unk_token_id)

None <|endoftext|> <|endoftext|>
None 0 0


In [6]:
# config for smollm 360M
tokenizer.padding_side = "left"
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id

In [7]:
print(tokenizer.pad_token, tokenizer.eos_token, tokenizer.unk_token)
print(tokenizer.pad_token_id, tokenizer.eos_token_id, tokenizer.unk_token_id)

<|endoftext|> <|endoftext|> <|endoftext|>
0 0 0


In [8]:
data = []
with open('event_text_mapping.jsonl', 'r') as file:
    for line in file:
        data.append(json.loads(line))
        # print(data)  # Each line is a dictionary


In [9]:
len(data)

792

In [10]:
data[0]

{'event_text': 'Late night study session at the café on 15th, Dec 2024 at 9:00 pm for 2 hours.',
 'output': {'action': 'study session',
  'date': '15/12/2024',
  'time': '9:00 PM',
  'attendees': None,
  'location': 'café',
  'duration': '2 hours',
  'recurrence': None,
  'notes': None}}

# process the dataset

In [11]:
def process_data(row):
    query_text = "<user>" + row["event_text"] + "</user>"
    tool_text = "<output>" + json.dumps(row["output"]) + "</output>"
    row["text"] = query_text + tool_text + tokenizer.eos_token
    return row

In [12]:
dataset = Dataset.from_list(data)

In [13]:
len(dataset)

792

In [14]:
dataset[0]

{'event_text': 'Late night study session at the café on 15th, Dec 2024 at 9:00 pm for 2 hours.',
 'output': {'action': 'study session',
  'attendees': None,
  'date': '15/12/2024',
  'duration': '2 hours',
  'location': 'café',
  'notes': None,
  'recurrence': None,
  'time': '9:00 PM'}}

In [15]:
dataset = dataset.map(
    process_data,
    num_proc= multiprocessing.cpu_count(),
    load_from_cache_file=False,
)

Map (num_proc=12):   0%|          | 0/792 [00:00<?, ? examples/s]

In [16]:
dataset[0]["text"]

'<user>Late night study session at the café on 15th, Dec 2024 at 9:00 pm for 2 hours.</user><output>{"action": "study session", "attendees": null, "date": "15/12/2024", "duration": "2 hours", "location": "caf\\u00e9", "notes": null, "recurrence": null, "time": "9:00 PM"}</output><|endoftext|>'

# split into train and test

In [17]:
train_valid = dataset.train_test_split(test_size=0.2, seed=42)

# load the base model

In [18]:
model = AutoModelForCausalLM.from_pretrained(checkpoint, device_map="cuda", torch_dtype=torch.float16)

config.json:   0%|          | 0.00/725 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.45G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [19]:
token_len = []
for val in dataset["text"]:
    token_len.append(len(tokenizer.encode(val, return_tensors="pt")[0]))

## average character length statistic across the whole dataset

In [20]:
min(token_len), max(token_len), sum(token_len) // len(token_len)

(94, 140, 112)

## prepare the model for lora based finetuning

In [21]:
model.config.use_cache = False
model.config.pretracining_tp = 1
model.gradient_checkpointing_enable()

model = prepare_model_for_kbit_training(model)
print(model.num_parameters()/ (10 ** 6), "M params")

361.82112 M params


# set the LoRA config

In [22]:
# LoRA config for Smollm 360M
peft_config = LoraConfig(
    r=32,
    lora_alpha=64,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM",
)

In [23]:
# Apply LoRA to the model
lora_model = get_peft_model(model, peft_config)

# Function to count parameters in LoRA layers
def count_lora_params(model):
    lora_params = 0
    total_params = 0
    for name, param in model.named_parameters():
        total_params += param.numel()
        if 'lora' in name:
            lora_params += param.numel()
    return lora_params, total_params

# Count the parameters
lora_params, total_params = count_lora_params(lora_model)
print(f"LoRA Parameters: {lora_params / (10 ** 6)} M Params")

LoRA Parameters: 17.36704 M Params


In [24]:
print(lora_model.num_parameters() / (10 ** 6), "M params")

379.18816 M params


In [25]:
use_bf16 = torch.cuda.is_bf16_supported()

# set the training config

In [27]:
# Set training parameters
training_arguments = SFTConfig(
    output_dir="./expt_results",
    num_train_epochs=50,
    per_device_train_batch_size=64,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=100,
    logging_steps=50,
    learning_rate=4e-4,
    weight_decay=0.001,
    fp16=not use_bf16,
    bf16=use_bf16,
    max_grad_norm=0.3,
    max_steps=-1, # the total number of training steps to perform
    warmup_ratio=0.3,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="tensorboard",
    dataset_text_field="text",
    max_seq_length=200,
    packing=True
)

In [28]:
trainer = SFTTrainer(
    model=model,
    train_dataset=train_valid["train"],
    eval_dataset=train_valid["test"],
    peft_config=peft_config,
    args=training_arguments
)

/usr/local/lib/python3.11/dist-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/tuners/tuners_utils.py:190: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/trl/trainer/sft_trainer.py:412: UserWarning: Padding-free training is enabled, but the attention implementation is not set to 'flash_attention_2'. Padding-free training flattens batches into a single sequence, and 'flash_attention_2' is the only known attention mechanism that reliably supports this. Using other implementations may lead to unexpected behavior. To ensure compatibility, set `attn_implementation='flash_attention_2'` in the model configurat

Adding EOS to train dataset:   0%|          | 0/633 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/633 [00:00<?, ? examples/s]

Packing train dataset:   0%|          | 0/633 [00:00<?, ? examples/s]

Adding EOS to eval dataset:   0%|          | 0/159 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/159 [00:00<?, ? examples/s]

Packing eval dataset:   0%|          | 0/159 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


# train the model

In [29]:
trainer.train()

Step,Training Loss
50,1.825600
100,0.522000
150,0.355300
200,0.283200
250,0.227900
300,0.179000
350,0.136900
400,0.108800
450,0.091500
500,0.077800


TrainOutput(global_step=500, training_loss=0.38078844690322877, metrics={'train_runtime': 874.1226, 'train_samples_per_second': 33.519, 'train_steps_per_second': 0.572, 'total_flos': 7080910574496000.0, 'train_loss': 0.38078844690322877})

In [30]:
import shutil

In [33]:
shutil.make_archive('/content/expt_results/checkpoint-300', 'zip', '/content/expt_results/checkpoint-300')

'/content/expt_results/checkpoint-300.zip'